# NRGV Data Analyst Interview Project

## Data Munging

### Future Work - Ideas

1) Could test for count of underscores in file name to chack for additional encodings<br>
2) Using LAT/LONG and Date, determine sunset/sunrise; progamaticaly remove records when the sun is not out (from sunset to sunrise).<br>
2a) Eliminate these records during the injest stage -Document time-delta vs lines of code vs reduced records.<br>
3) Automate the detection of new state in dictionary.

### Notebook Objectives:

<ul>
    <li>Data Reduction: Remove non daylight data.</li>
    <li>Write Record_Reduction_Statistics Log file.</li>
</ul>

-----------------------------

### Libraries

In [1]:
import requests as req     # DOWNLOADING URL VIA URL
import zipfile             # DEALING WITH ZIP COMPRESSED FILE
from io import BytesIO     # READING FILE FROM BUFFER
import os                  # DIRECTORY CREATION
import pandas as pd        # PANDAS DATAFRAME
import datetime            # DATE AND TIME
from datetime import datetime

### Variables

In [2]:
# PATH TO PLACE DATA
data_path = 'C://Users//bruce//Documents//Data//NRGV'

# SWSD_SUMMARY_DATA 
sumry_data_path = 'C://Users//bruce//Documents//Data//SWSD_SUMMARY_DATA'

# SWSD RECORDS REDUCTION
record_reduction_path_filename = sumry_data_path + "//Record_Reduction_Statistics.txt"

READ SWSD FILES

In [3]:
# 1) DETERMINE WHAT FILES ARE IN THE SWSD_SUMMARY_DATA DIRECTORY
# 2) ITERATE LIST AND READ FILES
# 3) MUNGE DATA
#    3a) Remove NIGHT records (sun will never shine at night) 
#        - have 0 Power(MW)
#          - find last non-zero value, by day
#          - find first non-zero value, by day
#          - between the hours of 


In [9]:
# INTERAGATE DIRECTORY FOR ALL FILES, ANY THAT ENDSWITH csv WILL BE PROCESSED FOR RECORD REDUCTION 
#  RECORD REDUCTION BASED UPON TIME OF DAY USING THE .between_time() FUNCTION

for smry_data_file in os.listdir(sumry_data_path):
    
    # ACT ONLY ON FILES THAT ENDSWITH .csv
    if smry_data_file.endswith(".csv"):
              
        # PROCESSING STATS
        start_time = datetime.now()
                
        # READ THE CURRENT SUMMARY DATA FILE
        swsd_smry_data = pd.read_csv(sumry_data_path + "//"+ smry_data_file, header=0, parse_dates=['LocalTime'],index_col=0)
        # 
        
        # DROP THE INDEX COLUMN
        #swsd_smry_data = swsd_smry_data.drop('Unnamed: 0', axis=1)
        
        # FILTER DATA BETWEEN DAYTIME (not_dark: 04:30:00 - 20:00:00) and NIGHTTIME (dark: 20:00:00 - 04:00:00)
        dark = swsd_smry_data.between_time('20:00:00', '04:30:00', include_start=False)
        not_dark = swsd_smry_data.between_time('04:30:00','20:00:00', include_start=False)
        
        # PROCESSING STATS
        stop_time = datetime.now()
        processing_elapsed_time = stop_time - start_time
        
        # PROCESSING STATISTICS
        state_name = smry_data_file[0:2].upper()
        start_time_txt = "The process started at: {}".format(start_time)
        stop_time_txt = "The process finished at: {}".format(stop_time)
        processing_elapsed_time_txt = "The processing elapsed time was: {}".format(processing_elapsed_time)
        state_name_txt = "The State processed was {}".format(state_name)
        base_rec_cnt = "The total number of records in the base data is: {:,}".format(len(swsd_smry_data))
        dark_rec_cnt = "The total number of records in the based data that belong to dark: {:,}".format(len(dark))
        light_rec_cnt = "The total number of records in the base data that belong to daylight:{:,}".format(len(not_dark))
        light_and_dark_rec_cnt = "The total number of records in the daylight and dark data is: {:,}".format(len(dark)+len(not_dark))
        light_total_power = "The total Power(MW) of daylight hours is: {:,.2f}".format(not_dark['Power(MW)'].sum())
        dark_total_power =  "The total Power(MW) of non-daylight hours is: {:,.2f}".format(dark['Power(MW)'].sum())
        match_test = len(swsd_smry_data) == (len(dark) + len(not_dark))
        match = "Do the the total number of records Match between base data and the sum of dark and daylight: {:,}".format(match_test)
        new_line = '\n'
        
        # PRINT PROCESSING STATISTICS TO SCREEN
        print(start_time_txt,new_line,stop_time_txt,new_line,processing_elapsed_time_txt,new_line,state_name_txt,\
              new_line,base_rec_cnt,new_line,dark_rec_cnt,new_line,light_rec_cnt,\
              new_line,light_and_dark_rec_cnt,new_line,match,new_line,light_total_power,new_line,dark_total_power)
        
        # CAPTURE PROCESSING STATISTICS FOR TXT FILE
        data_reduction = [str(start_time),str(stop_time),str(processing_elapsed_time),state_name_txt,base_rec_cnt,\
                          dark_rec_cnt,light_rec_cnt,light_and_dark_rec_cnt,match,light_total_power,dark_total_power]
        
        # WRITE NEW <STATE>_SWSD_SUMMARY_DATA_DAYTIME.csv
        not_dark.to_csv(sumry_data_path + "//" + state_name + "_SWSD_SUMMARY_DATA_DAYTIME.csv", \
                        header=True, index=True)
        
        # MEMORALIZE PROCESSING STATISTICS
        with open(record_reduction_path_filename, 'a') as new_file:
            for line in data_reduction:
                new_file.write(line)
                new_file.write('\n')

The process started at: 2022-10-16 15:39:01.142966 
 The process finished at: 2022-10-16 15:59:22.926897 
 The processing elapsed time was: 0:20:21.783931 
 The State being processed is AL 
 The total number of records in the base data is: 16,801,680 
 The total number of records in the based data that belong to dark: 5,900,590 
 The total number of records in the base data that belong to daylight:10,901,090 
 The total number of records in the daylight and dark data is: 16,801,680 
 Do the the total number of records Match between base data and the sum of dark and daylight: 1 
 The total Power(MW) of daylight hours is: 106,926,981.70 
 The total Power(MW) of daylight hours is: 0.00
The process started at: 2022-10-16 16:00:13.536175 
 The process finished at: 2022-10-16 16:25:05.083137 
 The processing elapsed time was: 0:24:51.546962 
 The State being processed is AZ 
 The total number of records in the base data is: 20,971,440 
 The total number of records in the based data that belo